In [161]:
import sqlite3
import pandas as pd

In [433]:
sql_query = """SELECT name FROM sqlite_master
    WHERE type='table';"""

In [442]:
professor_df = pd.read_excel("/Users/99ksh/TDB/professor.xlsx")
course_classed_df = pd.read_excel("/Users/99ksh/TDB/course_classed.xlsx")
norm_course_df = pd.read_excel("/Users/99ksh/TDB/norm_course.xlsx")
charge_df = pd.read_excel("/Users/99ksh/TDB/charge.xlsx")
university_df = pd.read_excel("/Users/99ksh/TDB/university.xlsx")

In [443]:
professor_df = professor_df.iloc[:, 1:]
course_classed_df = course_classed_df.iloc[:, 1:]
norm_course_df = norm_course_df.iloc[:, 1:]
charge_df = charge_df.iloc[:, 1:]
university_df = university_df.iloc[:, 1:]

In [444]:
university_df.rename(columns={"사립/국립 구분": "구분"}, inplace=True)
university_df.rename(columns={"대학소재지(시/도)": "대학소재지"}, inplace=True)

In [445]:
con = sqlite3.connect('kocw.db')

In [446]:
cur = con.cursor()
cur.execute(sql_query)
con.execute("PRAGMA foreign_keys = ON")
print(cur.fetchall())

[]


In [447]:
cur.execute("DROP TABLE IF EXISTS charge")
cur.execute("DROP TABLE IF EXISTS norm_course")
cur.execute("DROP TABLE IF EXISTS course_classed")
cur.execute("DROP TABLE IF EXISTS professor")
cur.execute("DROP TABLE IF EXISTS university")

In [448]:
university = """ CREATE TABLE university (
            제공대학명 CHAR(50) PRIMARY KEY NOT NULL UNIQUE,
            대학소재지 CHAR(50) ,
            구분 CHAR(50)
        ); """

In [449]:
professor = """ CREATE TABLE professor (
            교수코드 INTEGER PRIMARY KEY NOT NULL UNIQUE,
            담당교수명 CHAR(50) ,
            담당분야 CHAR(50),
            소속대학명 CHAR(50),
            FOREIGN KEY (소속대학명) REFERENCES university (제공대학명)
                ON UPDATE CASCADE
                ON DELETE SET NULL
        ); """

In [450]:
norm_course = """ CREATE TABLE norm_course (
    강의코드 INTEGER PRIMARY KEY NOT NULL UNIQUE,
    강의명 CHAR(50),
    강의년도 INTEGER,
    강의학기 INT,
    제공대학명 CHAR(50),
    제공일자 DATE,
    FOREIGN KEY (제공대학명) REFERENCES university (제공대학명)
    
        ); """

In [451]:
course_classed  = """ CREATE TABLE course_classed (
    강의명 CHAR(50) PRIMARY KEY NOT NULL UNIQUE,
    교육분류 CHAR(50)
        ); """

In [452]:
charge = """ CREATE TABLE charge (
    교수코드 INTEGER,
    강의코드 INTEGER,
    PRIMARY KEY (교수코드, 강의코드),
    FOREIGN KEY (교수코드) REFERENCES professor (교수코드)
        ON UPDATE CASCADE
        ON DELETE CASCADE,
    FOREIGN KEY (강의코드) REFERENCES norm_course (강의코드)
        ON UPDATE CASCADE
        ON DELETE CASCADE
        ); """

In [453]:
cur.execute(university)
print(university_df.itertuples())
for row in university_df.itertuples():
    university_1 = "INSERT INTO university(제공대학명, 대학소재지, 구분) VALUES (?, ?, ?)"
    cur.execute(university_1, (row[1], row[2], row[3]))
con.commit()
#cur.execute('DELETE FROM target_db')
cur.execute('SELECT * FROM university')

In [454]:
cur.execute(professor)
print(professor_df.itertuples())
for row in professor_df.itertuples():
    professor_1 = "INSERT INTO professor(교수코드, 담당교수명, 담당분야, 소속대학명) VALUES (?, ?, ?, ?)"
    cur.execute(professor_1, (row[1], row[2], row[3], row[4]))
con.commit()
cur.execute('SELECT * FROM professor')

In [455]:
cur.execute(norm_course)
print(norm_course_df.itertuples())
for row in norm_course_df.itertuples():
    norm_course_1 = "INSERT INTO norm_course(강의코드, 강의명, 강의년도, 강의학기, 제공대학명, 제공일자) VALUES (?, ?, ?, ?, ?, ?)"
    cur.execute(norm_course_1, (row[1], row[2], row[3], row[4], row[5], row[6]))
con.commit()
cur.execute('SELECT * FROM norm_course')
#cur.fetchall()

In [456]:
cur.execute(course_classed)
print(course_classed_df.itertuples())
for row in course_classed_df.itertuples():
    course_classed_1 = "INSERT INTO course_classed(강의명, 교육분류) VALUES (?, ?)"
    cur.execute(course_classed_1, (row[1], row[2]))
con.commit()
cur.execute('SELECT * FROM course_classed')
#cur.fetchall()

In [457]:
cur.execute(charge)
print(charge_df.itertuples())
for row in charge_df.itertuples():
    charge_1 = "INSERT INTO charge(교수코드, 강의코드) VALUES (?, ?)"
    cur.execute(charge_1, (row[1], row[2]))
con.commit()
cur.execute('SELECT * FROM charge')
#cur.fetchall()

In [459]:
cur.execute(sql_query)
cur.fetchall()

[('university',),
 ('professor',),
 ('norm_course',),
 ('course_classed',),
 ('charge',)]

In [327]:
conn.close

<function Connection.close()>

In [460]:
rows = cur.execute("SELECT name FROM sqlite_master WHERE type = 'table'")
tables = [row[0] for row in rows]
print(tables)

def sql_identifier(s):
    return '"' + s.replace('"', '""') + '"'

for table in tables:
    print("table: " + table)
    rows = cur.execute("PRAGMA table_info({})".format(sql_identifier(table)))
    print(rows.fetchall())
    rows = cur.execute("PRAGMA foreign_key_list({})".format(sql_identifier(table)))
    print(rows.fetchall())

['university', 'professor', 'norm_course', 'course_classed', 'charge']
table: university
[(0, '제공대학명', 'CHAR(50)', 1, None, 1), (1, '대학소재지', 'CHAR(50)', 0, None, 0), (2, '구분', 'CHAR(50)', 0, None, 0)]
[]
table: professor
[(0, '교수코드', 'INTEGER', 1, None, 1), (1, '담당교수명', 'CHAR(50)', 0, None, 0), (2, '담당분야', 'CHAR(50)', 0, None, 0), (3, '소속대학명', 'CHAR(50)', 0, None, 0)]
[(0, 0, 'university', '소속대학명', '제공대학명', 'CASCADE', 'SET NULL', 'NONE')]
table: norm_course
[(0, '강의코드', 'INTEGER', 1, None, 1), (1, '강의명', 'CHAR(50)', 0, None, 0), (2, '강의년도', 'INTEGER', 0, None, 0), (3, '강의학기', 'INT', 0, None, 0), (4, '제공대학명', 'CHAR(50)', 0, None, 0), (5, '제공일자', 'DATE', 0, None, 0)]
[(0, 0, 'university', '제공대학명', '제공대학명', 'NO ACTION', 'NO ACTION', 'NONE')]
table: course_classed
[(0, '강의명', 'CHAR(50)', 1, None, 1), (1, '교육분류', 'CHAR(50)', 0, None, 0)]
[]
table: charge
[(0, '교수코드', 'INTEGER', 0, None, 1), (1, '강의코드', 'INTEGER', 0, None, 2)]
[(0, 0, 'norm_course', '강의코드', '강의코드', 'CASCADE', 'CASCADE', 'NON